In [ ]:
import os 
from pdf2image import convert_from_path
import cv2
import pytesseract
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
def procesar_pdf(pdf_path):
    #Se encarga de convertir todo el pdf en imagenes, procesa cada imagen y extrae el texto 
    try:
        pages = convert_from_path(pdf_path, dpi = 400)
        texts = []

        for i, page in enumerate(pages):
            #guarda imagen como temporal
            temp_img_path = pdf_path.replace('.pdf', f'_page_{i}.jpg')
            page.save(temp_img_path, 'JPG')

            #carga imagen y procesa con cv2
            img = cv2.imread(temp_img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BAYER_BG2GRAY)
            thresh = cv2.threshold(gray,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

            #extrae con tesseract
            custom_config = r'--oem 3 --psm 6'
            text = pytesseract.image_to_string(thresh, config=custom_config, lang='eng')
            texts.append(text)

            #eliminar imagen temporal
            os.remove(temp_img_path)

        return "\n".join(texts)
    except Exception as e:
        return f"Error procesando {pdf_path} : {e}"